In [1]:
#import libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import RidgeClassifierCV
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score,classification_report
import sklearn.impute as imputers
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

In [4]:
from sklearn.utils import all_estimators
estimators = all_estimators(type_filter='classifier')
classification_estimators = []
i = 1
for name, class_ in estimators:
    classification_estimators.append(class_.__name__)
print(len(classification_estimators))

41


In [2]:
#load data
df = pd.read_csv('var_of_interest.csv')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
df.head()
df["PostCond"].value_counts()

1    62257
0    21063
Name: PostCond, dtype: int64

In [4]:
# #Splitting the dataset into the Training set and Test set
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [5]:
skf = StratifiedKFold(n_splits = 10, shuffle=True, random_state=42)

indices = skf.split(X, y)
X_train = dict()
y_train = dict()
X_test = dict()
y_test = dict()
cnt = 0
from sklearn.preprocessing import StandardScaler

for train_i, test_i in indices:
    X_train_i = X[train_i]
    X_test_i = X[test_i]
    y_train_i = y[train_i]
    y_test_i = y[test_i]

    sc1 = StandardScaler()
    imputer = imputers.IterativeImputer(random_state=42)
    
    X_train_i = imputer.fit_transform(X_train_i)
    X_train_i = sc1.fit_transform(X_train_i)
    X_test_i = imputer.transform(X_test_i)
    X_test_i = sc1.transform(X_test_i)
    
    X_train[cnt] = X_train_i
    X_test[cnt] = X_test_i
    y_train[cnt] = y_train_i
    y_test[cnt] = y_test_i

    cnt += 1


In [5]:
# #Scaling the Dataset
# from sklearn.preprocessing import StandardScaler
# sc1 = StandardScaler()
# sc2 = StandardScaler()
# imputer = imputers.IterativeImputer(random_state=42)
# # X_train = sc1.fit_transform(X_train)
# X_train = imputer.fit_transform(X_train)
# X_train = sc2.fit_transform(X_train)
# # X_test = sc1.transform(X_test)
# X_test = imputer.transform(X_test)
# X_test = sc2.transform(X_test)

In [6]:
models=[AdaBoostClassifier,BaggingClassifier,BernoulliNB,CalibratedClassifierCV,DecisionTreeClassifier,DummyClassifier,
        ExtraTreeClassifier,ExtraTreesClassifier,GaussianNB,GradientBoostingClassifier,HistGradientBoostingClassifier,KNeighborsClassifier,
        LinearDiscriminantAnalysis,LinearSVC,LogisticRegression,LogisticRegressionCV,MLPClassifier,
        NearestCentroid,PassiveAggressiveClassifier,Perceptron,QuadraticDiscriminantAnalysis,
        RandomForestClassifier,RidgeClassifier,RidgeClassifierCV,SGDClassifier,SVC] 

cross_val_models = dict()
for i in range(10):
        cross_val_models[i] = list()
        for model in models:
                cross_val_models[i].append(model)
            
#CategoricalNB
#ClassifierChain
#ComplementNB
#GaussianProcessClassifier
#LabelPropagation
#LabelSpreading
#MultinomialNB
#NuSVC
#OneVsOneClassifier
#OneVsRestClassifier,
#OutputCodeClassifier
#RadiusNeighborsClassifier
#StackingClassifier
#VotingClassifier

In [7]:
#Fitting the Model
from statistics import mean
from sklearn.metrics import balanced_accuracy_score, recall_score, precision_score, f1_score
model_accuracies =[]
model_recalls = []
model_precisions = []
model_f1 = []

for i in range(len(models)):
    accuracies = list()
    recalls = list()
    precisions = list()
    f1 = list()
    for k in X_train.keys():
        model = cross_val_models[k][i]()
        model.fit(X_train[k], y_train[k])
        y_pred= model.predict(X_test[k])
        accuracies.append(round(balanced_accuracy_score(y_test[k], y_pred),4))
        recalls.append(round(recall_score(y_test[k], y_pred), 4))
        precisions.append(round(precision_score(y_test[k], y_pred), 4))
        f1.append(round(f1_score(y_test[k], y_pred), 4))

    model_accuracies.append(mean(accuracies))
    model_recalls.append(mean(recalls))
    model_precisions.append(mean(precisions))
    model_f1.append(mean(f1))

    print (f'model : {models[i]} and  accuracy score is : {mean(accuracies)}, recall: {mean(recalls)}, precision: {mean(precisions)}, f1: {mean(f1)}')
    
result_df1 = pd.DataFrame({'Accuracy':model_accuracies, 'Recall':model_recalls, 'Precision':model_precisions, 'f1':model_f1}, index=models)
result_df1

model : <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'> and  accuracy score is : 0.50022, recall: 0.99921, precision: 0.74727, f1: 0.85508
model : <class 'sklearn.ensemble._bagging.BaggingClassifier'> and  accuracy score is : 0.55799, recall: 0.8593500000000001, precision: 0.77361, f1: 0.81424
model : <class 'sklearn.naive_bayes.BernoulliNB'> and  accuracy score is : 0.5, recall: 1.0, precision: 0.74717, f1: 0.8553
model : <class 'sklearn.calibration.CalibratedClassifierCV'> and  accuracy score is : 0.50015, recall: 0.99978, precision: 0.7472599999999999, f1: 0.8552799999999999
model : <class 'sklearn.tree._classes.DecisionTreeClassifier'> and  accuracy score is : 0.54592, recall: 0.75203, precision: 0.77101, f1: 0.76139
model : <class 'sklearn.dummy.DummyClassifier'> and  accuracy score is : 0.5, recall: 1.0, precision: 0.74717, f1: 0.8553
model : <class 'sklearn.tree._classes.ExtraTreeClassifier'> and  accuracy score is : 0.55238, recall: 0.75996, precision: 0.77418, f